In [27]:
# https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

import math
from math import *
from numpy import *
import pandas as pd
import numpy as np
import time


# import xlrd
# loc = ("C:\Users\thomp\Documents\IUP Overview")
# wb = xlrd.open_workbook(loc)
# sheet = wb.sheet_by_index(0)
# sheet.cell_value(0, 0)

# Haversine Formula:
def haversine(lon1, lat1, lon2, lat2):
    try:
        # convert decimal degrees to radians 
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        r = 6371 # 6371 Radius of earth in kilometers. Use 3956 for miles, 20887680 for feet
        return c * r
    except:
        return -1

# Hospitals: long / Lat:
Fajardo = [-65.65012, 18.33211]     # These have been updated from what was given
San_Pablo = [-65.65485, 18.33673]
San_Juan = [-66.06898, 18.44424]
Bayamon =  [-66.16327, 18.39657]
Arecibo = [-66.73172, 18.46705]

# Center point for San Pablo / Fajardo
SP_F_dropcenter = [-65.65046,18.33129]



# Drones V/MPC/Speed (km/h)/FT no cargo/Range no cargo/ Range w/ max cargo
DroneA = [50625.0, 3.5, 40.0, 35.0, 23.333,22.16667]
DroneB = [19800.0, 8.0, 79.0, 40.0, 52.667,47.4]
DroneC = [90000.0, 14.0, 64.0, 35.0, 37.333,29.86667]
DroneD = [12500.0, 11.0, 60.0, 18.0, 18,12.5]
DroneE = [13500.0, 15.0, 60.0, 15.0, 15,7.5]
DroneF = [40000.0, 22.0, 79.0, 24.0, 31.6,17.11667]
DroneG = [17408.0, 20.0, 64.0, 16.0, 17.066,6.4]

#Converting Lat/Lon to Cart:
def cartesian(self,longitude,latitude, elevation):
    R = 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(longitude) * math.sin(latitude)
    Y = R * math.sin(longitude) * math.sin(latitude)
    Z = R * math.cos(latitude)

def cartesian3(self,longitude,latitude, elevation):
    X = longitude * 60 * 1852 * math.cos(latitude)
    Y = latitude * 60 * 1852
    Z = elevation
    return X,Y,Z

# Area Search for San Juan and Bayamon: Long / Lat
NW1 = [-66.210, 18.455]
SW1 = [-66.210, 18.267]
NE1 = [-65.960, 18.455]
SE1 = [-65.690, 18.267]
Point1 = [-66.210, 18.267]

# Boundary limits when searching though the loop
i = Point1[0]
j = Point1[1]

#Loop for San Juan and Bayamon
Haver_out1 = pd.DataFrame(columns = ['Lats','Lons','San Juan','Bayamon','Valid Points'])
san_juan = []
bayamon = []
lats1 = []
lons1 = []
for i in np.arange(NW1[0], NE1[0],0.001):
    for j in np.arange(SE1[1], NE1[1],0.001):
        r_1 = haversine(float(San_Juan[0]), float(San_Juan[1]), i, j)
        r_2 = haversine(float(Bayamon[0]), float(Bayamon[1]), i, j)
        san_juan.append(r_1)
        bayamon.append(r_2)
        lats1.append(j)
        lons1.append(i)
        #print ('r_1 = ',r_1,  ' and r_2 = ',r_2)
        #Haver_out.loc[j, ['San Juan']] = r_1
        #Haver_out.loc[j, ['Bayamon']] = r_2
        #Haver_out.loc[j, ['Lon']] = i
Haver_out1['Lats'] = lats1
Haver_out1['Lons'] = lons1
Haver_out1['San Juan'] = san_juan
Haver_out1['Bayamon'] = bayamon

 # print Haver_out1






         Lats    Lons   San Juan    Bayamon Valid Points
0      18.267 -66.210  24.696407  15.228453          NaN
1      18.268 -66.210  24.607736  15.123291          NaN
2      18.269 -66.210  24.519250  15.018215          NaN
3      18.270 -66.210  24.430948  14.913228          NaN
4      18.271 -66.210  24.342835  14.808332          NaN
5      18.272 -66.210  24.254911  14.703528          NaN
6      18.273 -66.210  24.167179  14.598819          NaN
7      18.274 -66.210  24.079640  14.494207          NaN
8      18.275 -66.210  23.992298  14.389693          NaN
9      18.276 -66.210  23.905154  14.285280          NaN
10     18.277 -66.210  23.818210  14.180971          NaN
11     18.278 -66.210  23.731468  14.076766          NaN
12     18.279 -66.210  23.644932  13.972670          NaN
13     18.280 -66.210  23.558602  13.868684          NaN
14     18.281 -66.210  23.472481  13.764810          NaN
15     18.282 -66.210  23.386572  13.661052          NaN
16     18.283 -66.210  23.30087

In [3]:
#loop to get valid points
k = 0
Vpoints =[]

Haver_out1 = Haver_out1

for k in range(len(Haver_out1)):
    if DroneC[5]/2 - Haver_out1.iloc[k]['San Juan'] >= 0 and DroneC[5]/2 - Haver_out1.iloc[k]['Bayamon'] >=0:
        Vpoints.append([Haver_out1.iloc[k]['Lats'], Haver_out1.iloc[k]['Lons']])
    else:
        #Haver_out1.drop(Haver_out1.index[k])
        Vpoints.append(np.nan)

Haver_out1['Valid Points'] = Vpoints
Haver_out1 = Haver_out1.dropna()
#Haver_out1[Haver_out1.Valid Points != '[NaN,NaN]'


# print Haver_out1

NameError: name 'Haver_out1' is not defined

In [0]:
# loop to get max sum(s)
Distance = 0
best_index = []

for k in range(len(Haver_out1)):
    Distance_calc = Haver_out1.iloc[k]['San Juan'] + Haver_out1.iloc[k]['Bayamon']
    if Distance_calc > Distance:
        Distance = Distance_calc
        best_index = Haver_out1.iloc[k]

print best_index


In [18]:
import math
from math import *

# Drones
# volume / MPL / speed / flight time / range no cargo/ range with cargo
A = [50625.0, 3.5, 40.0, 35.0, 23.333,22.1667]
B = [19800.0, 8.0, 79.0, 40.0, 52.667,47.4]
C = [90000.0, 14.0, 64.0, 35.0, 37.333,29.8667]
D = [12500.0, 11.0, 60.0, 18.0, 18,12.5]
E = [13500.0, 15.0, 60.0, 15.0, 15,7.5]
F = [40000.0, 22.0, 79.0, 24.0, 31.6,17.11667]
G = [17408.0, 20.0, 64.0, 16.0, 17.066,6.4]
# H = [199875, 0, 0, na, 0]

#this function does not work
#A_range = A[3]*math.e**(-.15*A[1])
#B_range = B[3]*math.e**(-.15*B[1])
#C_range = C[3]*math.e**(-.15*C[1])
#D_range = D[3]*math.e**(-.15*D[1])
#E_range = E[3]*math.e**(-.15*E[1])
#F_range = F[3]*math.e**(-.15*F[1])
#G_range = G[3]*math.e**(-.15*G[1])

# Radius with Cargo use minus .5 minute per pound of cargo

A[5] = A[2]*((A[3]-A[1]*0.5)/60.0)
B[5] = B[2]*((B[3]-B[1]*0.5)/60.0)
C[5] = C[2]*((C[3]-C[1]*0.5)/60.0)
D[5] = D[2]*((D[3]-D[1]*0.5)/60.0)
E[5] = E[2]*((E[3]-E[1]*0.5)/60.0)
F[5] = F[2]*((F[3]-F[1]*0.5)/60.0)
G[5] = G[2]*((G[3]-G[1]*0.5)/60.0)

#Flight time with cargo
print (A[3]-A[1]*.5)
print (B[3]-B[1]*.5)
print (C[3]-C[1]*.5)
print (D[3]-D[1]*.5)
print (E[3]-E[1]*.5)
print (F[3]-F[1]*.5)
print (G[3]-G[1]*.5)

#print A_range
#print B_range
#print C_range
#print D_range
#print E_range
#print F_range
#print G_range
#print A[5]
#print B[5]
#print C[5]
#print D[5]
#print E[5]
#print F[5]
#print G[5]

33.2500000000000
36.0000000000000
28.0000000000000
12.5000000000000
7.50000000000000
13.0000000000000
6.00000000000000
